In [121]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
import sherpa
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


import time

import sys
sys.path.append('/home/yusukemh/github/yusukemh/StatisticalDownscaling/writeup')
from config import C_COMMON, C_GRID, C_SINGLE, FILENAME
from util import load_data, NeuralNetwork

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler


import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression as LR
from tqdm import tqdm

# enable autoreload
%load_ext autoreload
%autoreload 2

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import L1, L2

def define_model(
    input_dim=20,
    n_units=512,
    activation='selu',#selu
    learning_rate=0.00001,
    loss='mse',
    batch_size=64
):
    inputs = Input(shape=(input_dim))
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(inputs)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(x)
    x = Dropout(rate=0.5)(x)# serves as regularization
    outputs = Dense(units=1, activation='softplus')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        metrics=[RootMeanSquaredError()]
    )
    return model, batch_size


In [14]:
columns = C_SINGLE
n_run = 100

df_train, df_test = load_data(columns + C_COMMON, FILENAME)

In [15]:
n_model = NeuralNetwork(
    params={
        'n_units': 559,
        'learning_rate': 0.0004886843292039,
        'input_dim': 16,
        'batch_size': 256,
        'loss': 'mse'
    },
    columns=columns,
    model_func=define_model
)

In [17]:
result = n_model.evaluate(df_train, df_test)

  8%|▊         | 2/24 [01:40<18:23, 50.18s/it]


KeyboardInterrupt: 

In [34]:
# set up sherpa
parameters = [
    sherpa.Discrete('n_units', [256, 1024]),
    sherpa.Continuous('learning_rate', [0.00001, 0.01]),
    sherpa.Choice('batch_size', [64, 128, 192, 256, 512]),
    sherpa.Choice('loss', ['mse', 'mae'])
]
study = sherpa.Study(
    parameters=parameters,
    algorithm=sherpa.algorithms.RandomSearch(max_num_trials=n_run),
    lower_is_better=True
)

dfs = []
for i, trial in enumerate(study):
    # obtain hyperparameters
    params = {key: val for key, val in trial.parameters.items()}
    params['input_dim'] = len(columns)

    for skn in tqdm(df_train['skn'].unique()):
        model = NeuralNetwork(
            model_func=define_model,
            params=params,
            columns=columns
        )
        ret = model.cross_val_predict(df_train, skn)
        df = pd.DataFrame()
        df['trial_id'] = i,
        df['rmse'] = [ret['rmse']]
        df['mae'] = [ret['mae']]
        df['skn'] = [skn]
        for key, val in params.items():
            df[key] = [val]
        dfs.append(df)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running. Access via
http://10.100.11.207:8887 if on a cluster or
http://localhost:8887 if running locally.
-------------------------------------------------------


 * Serving Flask app 'sherpa.app.app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


100%|██████████| 24/24 [00:00<00:00, 338.10it/s]


# get the result

In [2]:
df_report = pd.read_csv('nn_report_500_single.csv')

In [3]:
df_groupby = df_report.groupby(by='trial_id').mean() 

In [5]:
objective = 'rmse'
idx_best_objective = df_groupby[objective].argmin()
df_report.query(f'trial_id == {idx_best_objective}').iloc[0][['n_units', 'learning_rate', 'input_dim', 'batch_size', 'loss']].to_dict()
# df_groupby.iloc[idx_best_objective]# [['n_units', 'learning_rate', 'input_dim', 'batch_size', 'loss']].to_dict()

{'n_units': 559,
 'learning_rate': 0.0004886843292039,
 'input_dim': 16,
 'batch_size': 256,
 'loss': 'mse'}

In [8]:
objective = 'mae'
idx_best_objective = df_groupby[objective].argmin()
df_report.query(f'trial_id == {idx_best_objective}').iloc[0][['n_units', 'learning_rate', 'input_dim', 'batch_size', 'loss']].to_dict()
# df_groupby.iloc[idx_best_objective]# [['n_units', 'learning_rate', 'input_dim', 'batch_size', 'loss']].to_dict()

{'n_units': 559,
 'learning_rate': 0.0004886843292039,
 'input_dim': 16,
 'batch_size': 256,
 'loss': 'mse'}

In [9]:
df_report.query(f'trial_id == {idx_best_objective}')

,Unnamed: 0,trial_id,rmse,mae,epochs,skn,n_units,learning_rate,batch_size,loss,input_dim
984,0,41,5.552019,3.459787,512,54.0,559,0.000489,256,mse,16
985,0,41,5.987925,3.813104,577,79.0,559,0.000489,256,mse,16
986,0,41,4.637724,2.477723,446,338.0,559,0.000489,256,mse,16
987,0,41,2.009247,1.385104,482,250.0,559,0.000489,256,mse,16
988,0,41,2.217039,1.503654,491,267.0,559,0.000489,256,mse,16
989,0,41,1.561271,0.734411,520,296.1,559,0.000489,256,mse,16
990,0,41,1.387866,0.748015,584,311.0,559,0.000489,256,mse,16
991,0,41,1.612915,0.939158,347,396.0,559,0.000489,256,mse,16
992,0,41,1.697855,1.060608,380,400.0,559,0.000489,256,mse,16
993,0,41,1.832052,1.212967,354,406.0,559,0.000489,256,mse,16


# Obtain the best hyperparameter for each station

In [4]:
df_report = pd.read_csv('nn_report_500_single.csv')

In [23]:
print('[')
for i, row in df_report.loc[df_report.groupby('skn')['rmse'].idxmin()].iterrows():
    temp = {
        'skn': row['skn'],
        'params': row[['n_units', 'learning_rate', 'input_dim', 'batch_size', 'loss']].to_dict()
    }
    print('\t', end='')
    print(temp, end=',\n')
print(']')

[
	{'skn': 54.0, 'params': {'n_units': 274, 'learning_rate': 0.001701746715659, 'input_dim': 16, 'batch_size': 192, 'loss': 'mse'}},
	{'skn': 79.0, 'params': {'n_units': 948, 'learning_rate': 0.0016483029312347, 'input_dim': 16, 'batch_size': 128, 'loss': 'mae'}},
	{'skn': 87.0, 'params': {'n_units': 315, 'learning_rate': 0.0018353200721248, 'input_dim': 16, 'batch_size': 512, 'loss': 'mse'}},
	{'skn': 250.0, 'params': {'n_units': 558, 'learning_rate': 0.001129450809096, 'input_dim': 16, 'batch_size': 512, 'loss': 'mse'}},
	{'skn': 267.0, 'params': {'n_units': 558, 'learning_rate': 0.001129450809096, 'input_dim': 16, 'batch_size': 512, 'loss': 'mse'}},
	{'skn': 296.1, 'params': {'n_units': 558, 'learning_rate': 0.001129450809096, 'input_dim': 16, 'batch_size': 512, 'loss': 'mse'}},
	{'skn': 311.0, 'params': {'n_units': 558, 'learning_rate': 0.001129450809096, 'input_dim': 16, 'batch_size': 512, 'loss': 'mse'}},
	{'skn': 338.0, 'params': {'n_units': 558, 'learning_rate': 0.0011294508090